# 张量运算

深度神经网络学到的所有变换都可以简化为数值数据张量上的一些**张量运算**（tensor operation）。

## 逐元素运算

relu 运算和加法都是逐元素（element-wise）运算，即该运算独立地应用于张量中的每个元素，这些运算非常适合大规模并行实现（向量化实现） 。如果你想对逐元素运算编写简单的 Python 实现，那么可以用for循环。

下面是对逐元素 relu 运算的简单实现

In [8]:
def native_relu(x):
    assert len(x.shape) == 2 # x 是一个 Numpy 2D 张量
    x = x.copy() # 避免覆盖输入张量
    for i in range(x.shape[0]):
        for j in range(x.shape[1]):
            x[i, j] = max(x[i, j], 0)
    return x

对于加法采用同样的实现方法。

In [7]:
def native_add(x, y):
    assert len(x.shape) == 2 # x 和 y 是 Numpy 的 2D 张量
    assert x.shape == y.shape

    x = x.copy()
    for i in range(x.shape[0]):
        for j in range(x.shape[1]):
            x[i, j] += y[i, j]
    
    return x

使用同样的方法可以实现逐元素的乘法、减法等。

在实践中处理 Numpy 数组时，这些运算都是优化好的 Numpy 内置函数，这些函数将大量运算交给安装好的基础线性代数子程序（BLAS，basic linear algebra subprograms）实现。BLAS是低层次的、高度并行的、高效的张量操作程序，通常用 Fortran 或 C 语言来实现。

因此在Numpy 中可以直接进行下列逐元素运算，速度非常快。

## 广播

广播包含两步：

1. 向较小的张量添加轴（叫做广播轴），使其 ndim 与较大的张量相同。
2. 将较小的张量沿着新轴重复，使其形状与较大的张量相同。